In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import requests
import time
import gmaps
import gmaps.datasets
from scipy.stats import linregress
# Import API key
from api_keys import gkey

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [2]:
# Census data files
census_state_path_2015 = "Output/2015_cleaned_state.csv"
census_state_code_data = "Output/county_code_data.csv"

In [3]:
# Read the Census data
census_state_results_2015 = pd.read_csv(census_state_path_2015)
census_state_code = pd.read_csv(census_state_code_data)


C:\Users\marti\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,1,2,3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
census_state_results_2015.head()

,State,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,...,OtherTrans,WorkatHome,MeanCommute,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Poverty,ChildPoverty,Unemployment
0,Alabama,4830620,2341093,2489527,193159,3204658,1270556,22007,59324,1089,...,20994,56260,25,1583312,329430,105863,3715,910480,1318475,454437
1,Alaska,733375,384160,349215,47860,457438,23611,98381,42323,8638,...,19750,16083,10,240061,88175,22190,634,74612,99617,61930
2,Arizona,6641928,3299088,3342840,2015055,3751927,263011,265835,193799,11587,...,77252,156279,21,2230882,412325,166404,5502,1210351,1724490,603924
3,Arkansas,2958208,1451913,1506295,203216,2177095,456609,16531,39968,6628,...,15580,40259,22,969630,205337,78614,2406,571519,821223,231142
4,California,38421464,19087135,19334329,14746502,14878698,2157932,144676,5192680,133245,...,437377,917120,24,13403787,2381720,1434085,34099,6260461,8451647,3845611


In [5]:
census_state_code.head()

,State Code,State,County,Region,Division,lat,lng
0,WA,Washington,Pierce,West,Pacific,46.9994,-122.3921
1,WA,Washington,Kittitas,West,Pacific,47.2507,-121.0989
2,WA,Washington,Lincoln,West,Pacific,47.3048,-117.9713
3,WA,Washington,Pierce,West,Pacific,47.3352,-122.5968
4,WA,Washington,Snohomish,West,Pacific,48.0639,-122.0920


In [6]:
census_2015_state_data = pd.merge(census_state_results_2015, census_state_code, how="left", on=["State","State"])
census_2015_state_data.head()


,State,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,...,FamilyWork,Poverty,ChildPoverty,Unemployment,State Code,County,Region,Division,lat,lng
0,Alabama,4830620,2341093,2489527,193159,3204658,1270556,22007,59324,1089,...,3715,910480,1318475,454437,AL,Madison,South,East South Central,34.8491,-86.5222
1,Alabama,4830620,2341093,2489527,193159,3204658,1270556,22007,59324,1089,...,3715,910480,1318475,454437,AL,Covington,South,East South Central,31.3006,-86.3975
2,Alabama,4830620,2341093,2489527,193159,3204658,1270556,22007,59324,1089,...,3715,910480,1318475,454437,AL,Clarke,South,East South Central,31.9111,-87.7419
3,Alabama,4830620,2341093,2489527,193159,3204658,1270556,22007,59324,1089,...,3715,910480,1318475,454437,AL,Blount,South,East South Central,33.8789,-86.8241
4,Alabama,4830620,2341093,2489527,193159,3204658,1270556,22007,59324,1089,...,3715,910480,1318475,454437,AL,Coosa,South,East South Central,32.9791,-86.0358


In [7]:
# census_2015 = census_state_results_2015[["State","TotalPop","Income","Professional","Service","Office","Construction","Production","code"]]
census_2015 = pd.DataFrame(census_2015_state_data)
census_2015.head()

,State,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,...,FamilyWork,Poverty,ChildPoverty,Unemployment,State Code,County,Region,Division,lat,lng
0,Alabama,4830620,2341093,2489527,193159,3204658,1270556,22007,59324,1089,...,3715,910480,1318475,454437,AL,Madison,South,East South Central,34.8491,-86.5222
1,Alabama,4830620,2341093,2489527,193159,3204658,1270556,22007,59324,1089,...,3715,910480,1318475,454437,AL,Covington,South,East South Central,31.3006,-86.3975
2,Alabama,4830620,2341093,2489527,193159,3204658,1270556,22007,59324,1089,...,3715,910480,1318475,454437,AL,Clarke,South,East South Central,31.9111,-87.7419
3,Alabama,4830620,2341093,2489527,193159,3204658,1270556,22007,59324,1089,...,3715,910480,1318475,454437,AL,Blount,South,East South Central,33.8789,-86.8241
4,Alabama,4830620,2341093,2489527,193159,3204658,1270556,22007,59324,1089,...,3715,910480,1318475,454437,AL,Coosa,South,East South Central,32.9791,-86.0358


In [8]:
gmaps.configure(api_key=gkey)
census_2015 = gmaps.datasets.load_dataset_as_census_2015("TotalPop")
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(census_2015[["lat","lng"]], weights=census_2015["Income"],max_intensity=30, point_radius=3.0)
fig.add_layer(heatmap_layer)
fig                                                


# locations = census_2015[["lat","lng"]]
# # city_data_df = city_data_df.dropna()
# population = census_2015["TotalPop"]

# heatmap = gmaps.heatmap_layer(locations,weights=population, dissipating=True, max_intensity = 100, point_radius=5)
# heatmap_layer.gradient = ["silver","gray","white"]
# fig.add_layer(heatmap_layer)
# fig

AttributeError: module 'gmaps.datasets' has no attribute 'load_dataset_as_census_2015'

In [ ]:
fig = gmaps.figure(map_type="HYBRID")
locations = census_2015[["lat","lng"]]
# city_data_df = city_data_df.dropna()
population = census_2015["TotalPop"]

heatmap = gmaps.heatmap_layer(locations,weights=population, dissipating=True, max_intensity = 100, point_radius=5)
heatmap_layer.gradient = ["silver","gray","white"]
fig.add_layer(heatmap_layer)
fig


In [ ]:
#Data Collection for Census data
total_pop = census_state_results_2015["TotalPop"].sum()
men = census_state_results_2015["Men"].sum()/total_pop * 100
women= census_state_results_2015["Women"].sum()/total_pop * 100
employed_num = census_state_results_2015["Employed"].sum()
employed = census_state_results_2015["Employed"].sum()/total_pop * 100
income_avg = census_state_results_2015["Income"].mean()
professional = census_state_results_2015["Professional"].sum()/employed_num * 100
service = census_state_results_2015["Service"].sum()/employed_num * 100
office = census_state_results_2015["Office"].sum()/employed_num * 100
construction = census_state_results_2015["Construction"].sum()/employed_num * 100
production = census_state_results_2015["Production"].sum()/employed_num * 100
drive = census_state_results_2015["Drive"].sum()/employed_num * 100
carpool = census_state_results_2015["Carpool"].sum()/employed_num * 100
transit = census_state_results_2015["Transit"].sum()/employed_num * 100
walk = census_state_results_2015["Walk"].sum()/employed_num * 100
workathome = census_state_results_2015["WorkatHome"].sum()/employed_num * 100
privatework = census_state_results_2015["PrivateWork"].sum()/employed_num * 100
publicwork = census_state_results_2015["PublicWork"].sum()/employed_num * 100
selfemployed = census_state_results_2015["SelfEmployed"].sum()/employed_num * 100
familywork = census_state_results_2015["FamilyWork"].sum()/employed_num * 100
poverty = census_state_results_2015["Poverty"].sum()/employed_num * 100
childpoverty = census_state_results_2015["ChildPoverty"].sum()/total_pop * 100
unemployment = census_state_results_2015["Unemployment"].sum()/total_pop * 100

In [ ]:
state_summary_2015_df = pd.DataFrame({"Total Pop": [total_pop],
                           "Men%": [men],
                           "Women%": [women],
                           "Income_Avg": [income_avg],
                           "Employed%": [employed],
                           "Professional%": [professional],
                           "Service%": [service],
                           "Office%": [office],
                           "Construction%": [construction],
                           "Production%": [production],
                           "Drive%": [drive],
                           "Carpool%": [carpool],
                           "Transit%": [transit],
                           "Walk%": [walk],
                           "WorkatHome%": [workathome],
                           "Privatework%": [privatework],
                           "Publicwork%": [publicwork],
                           "Selfemployed%": [selfemployed],
                           "Familywork%": [familywork],
                           "Poverty%": [poverty],
                           "Childpoverty%": [childpoverty],
                           "Selfemployed%": [selfemployed],
                           "Unemployment%": [unemployment]})           
                          
             
state_summary_2015_df .style.format({"Total Pop":"{:,}",
                         "Men%":"{:,.2f}%",
                         "Women%":"{:,.2f}%",
                         "Income_Avg":"${:,.2f}",
                         "Employed%":"{:,.2f}%",
                         "Professional%":"{:,.2f}%",
                         "Service%":"{:,.2f}%",
                         "Office%":"{:,.2f}%",
                         "Construction%":"{:,.2f}%",
                         "Production%":"{:,.2f}%",
                         "Drive%":"{:,.2f}%",
                         "Carpool%":"{:,.2f}%",
                         "Transit%":"{:,.2f}%",
                          "Walk%":"{:,.2f}%",
                          "WorkatHome%":"{:,.2f}%",
                          "Privatework%":"{:,.2f}%",
                          "Publicwork%":"{:,.2f}%",
                          "Selfemployed%":"{:,.2f}%",
                          "Familywork%":"{:,.2f}%",
                          "Poverty%":"{:,.2f}%",
                          "Childpoverty%":"{:,.2f}%",
                          "Selfemployed%":"{:,.2f}%",
                          "Unemployment%":"{:,.2f}%"})  

In [ ]:
plt.scatter(census_state_results_2015["Income"],census_state_results_2015["Professional"], marker="o", facecolors="red", edgecolors="black",alpha=0.5)
plt.xlabel("Income")
plt.ylabel("Professional")
plt.title(f"Avg Income vs Professional Sector")
# plt.savefig("../output_data//Avg Income vs Professional Sector.png")
plt.grid()
plt.show()

In [ ]:
x_value = census_state_results_2015["Income"]
y_value = census_state_results_2015["Professional"]


(slope, intercept, rvalue, pvalue, stderr) = linregress(x_value, y_value)

print(f"The r-squared is: {rvalue**2}")
r_line = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))

regression_values = x_value * slope + intercept
plt.scatter(x_value,y_value)
plt.annotate(r_line,(5, 40),color="red")
plt.plot(x_value,regression_values, "r-")
plt.title("Avg Income vs Professional Sector")
plt.xlabel("Income")
plt.ylabel("Professional Sector")
# plt.savefig("../output_data//Northern Hemisphere - Max Temp vs. Latitude Linear Regression.png")
plt.show()

In [ ]:
# plt.figure(figsize=(15, 10))
# plt.bar(x, y, color="blue", alpha=1, align="center")
# plt.xlim(-.75, len(x)+.75)
# plt.ylim(0, max(y)+25)
# plt.title("Avg Income vs Professional Sector")
# plt.xlabel("Income")
# plt.ylabel("Professional Sector")
# tick_locations = [value for value in x]
# plt.xticks(tick_locations, rotation=90)
# plt.show()

In [ ]:
list(census_2015)

In [ ]:
# income_data = census_2015[census_2015["State"]["Income"]
# professional_data = census_2015[census_2015["State"]["Professional"]
# service_data = census_2015[census_2015["State"]["Service"]
# office_data = census_2015[census_2015["State"]["Office"]
# construction_data = census_2015[census_2015["State"]["Construction"]
# productiondata = census_2015[census_2015["State"]["Production"]

In [ ]:
# # Determine outliers using upper and lower bounds

# # Capamulin Quartile data
# income_quartiles = income_data.quantile([.25,.5,.75])
# lowerq = income_quartiles[0.25]
# upperq = income_quartiles[0.75]
# iqr = upperq-lowerq
# print(f"Capomullin lower quartile is: {lowerq}")
# print(f"Capomullin upper quartile is: {upperq}")
# print(f"Capomullin interquartile range is: {iqr}")
# print(f"Capomullin is: {income_quartiles[0.5]} ")

# cap_lower_bound = lowerq - (1.5*iqr)
# cap_upper_bound = upperq + (1.5*iqr)
# print(f"Capomullin below {cap_lower_bound} could be outliers.")
# print(f"Capomullin above {cap_upper_bound} could be outliers.")


# # Ramicane Quartile data
# professional_quartiles = professional_data.quantile([.25,.5,.75])
# lowerq = professional_quartiles[0.25]
# upperq = professional_quartiles[0.75]
# iqr = upperq-lowerq
# print(f"Ramicane lower quartile is: {lowerq}")
# print(f"Ramicane upper quartile is: {upperq}")
# print(f"Ramicane interquartile range is: {iqr}")
# print(f"Ramicane is: {professional_quartiles[0.5]} ")

# ram_lower_bound = lowerq - (1.5*iqr)
# ram_upper_bound = upperq + (1.5*iqr)
# print(f"Ramicane below {ram_lower_bound} could be outliers.")
# print(f"Ramicane above {ram_upper_bound} could be outliers.")

# # Infubinol Quartile data
# service_quartiles = service_data.quantile([.25,.5,.75])
# lowerq = service_quartiles[0.25]
# upperq = service_quartiles[0.75]
# iqr = upperq-lowerq
# print(f"Infubinol lower quartile is: {lowerq}")
# print(f"Infubinol upper quartile is: {upperq}")
# print(f"Infubinol interquartile range is: {iqr}")
# print(f"Infubinol is: {service_quartiles[0.5]} ")

# inf_lower_bound = lowerq - (1.5*iqr)
# inf_upper_bound = upperq + (1.5*iqr)
# print(f"Infubinol below {inf_lower_bound} could be outliers.")
# print(f"Infubinol above {inf_upper_bound} could be outliers.")

# # Ceftamin Quartile data
# office_quartiles = office_data.quantile([.25,.5,.75])
# lowerq = office_quartiles[0.25]
# upperq = office_quartiles[0.75]
# iqr = upperq-lowerq
# print(f"Ceftamin lower quartile is: {lowerq}")
# print(f"Ceftamin upper quartile is: {upperq}")
# print(f"Ceftamin interquartile range is: {iqr}")
# print(f"Ceftamin is: {office_quartiles[0.5]} ")

# cef_lower_bound = lowerq - (1.5*iqr)
# cef_upper_bound = upperq + (1.5*iqr)
# print(f"Ceftamin below {cef_lower_bound} could be outliers.")
# print(f"Ceftamin above {cef_upper_bound} could be outliers.")


# capomulin_outlier = [out for out in capomulin_volume if out < cap_lower_bound or out > cap_upper_bound ]
# ramicane_outlier = [out for out in ramicane_volume if out < ram_lower_bound or out > ram_upper_bound ]
# infubinol_outlier = [out for out in infubinol_volume if out < inf_lower_bound or out > inf_upper_bound ]
# ceftamin_outlier = [out for out in ceftamin_volume if out < cef_lower_bound or out > cef_upper_bound ]
# print(f"The outlier for Capomullin is {capomulin_outlier}. There is no outliers for Capomulin")
# print(f"The outlier for Ramicane is {ramicane_outlier}. There is no outliers for Ramicane")
# print(f"The outlier for Infubinol is {infubinol_outlier}. There is no outliers for Infubinol")
# print(f"The outlier for Ceftamin is {ceftamin_volume}. There is no outliers for Ceftamin")
